# Pre-Processing

In [1]:
from google.colab import drive
drive.mount('/content/drive')
import warnings
warnings.filterwarnings('ignore')

Mounted at /content/drive


In [2]:
%cd /content/drive/My Drive/Rakamin FinPro/

/content/drive/My Drive/Rakamin FinPro


In [3]:
import pandas as pd
import numpy as np

In [4]:
df = pd.read_csv("SBAnational.csv", parse_dates=['ApprovalDate', 'DisbursementDate'])

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 899164 entries, 0 to 899163
Data columns (total 27 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   LoanNr_ChkDgt      899164 non-null  int64         
 1   Name               899150 non-null  object        
 2   City               899134 non-null  object        
 3   State              899150 non-null  object        
 4   Zip                899164 non-null  int64         
 5   Bank               897605 non-null  object        
 6   BankState          897598 non-null  object        
 7   NAICS              899164 non-null  int64         
 8   ApprovalDate       899164 non-null  datetime64[ns]
 9   ApprovalFY         899164 non-null  object        
 10  Term               899164 non-null  int64         
 11  NoEmp              899164 non-null  int64         
 12  NewExist           899028 non-null  float64       
 13  CreateJob          899164 non-null  int64   

### Mengatasi ApprovalFY yang memiliki niali string

In [6]:
def change_year(x):
    if x['ApprovalFY'] == '1976A':
        return 1976
    else:
        return int(x['ApprovalFY'])
    
df['ApprovalFY'] = df.apply(change_year,axis=1)

### Mengatasi DisbursementGross, BalanceGross, ChgOffPrinGr, GrAppv, SBA_Appv yang memiliki tanda '$' dan ','

In [7]:
def remove_sign(item):
        new = item.replace('$','')
        new2 = new.replace(' ','')
        new3 = new2.replace(',','')
        return float(new3)

df['DisbursementGross'] = df['DisbursementGross'].apply(lambda x : remove_sign(x))
df['BalanceGross'] = df['BalanceGross'].apply(lambda x : remove_sign(x))
df['ChgOffPrinGr'] = df['ChgOffPrinGr'].apply(lambda x : remove_sign(x))
df['GrAppv'] = df['GrAppv'].apply(lambda x : remove_sign(x))
df['SBA_Appv'] = df['SBA_Appv'].apply(lambda x : remove_sign(x))

### Mengambil 2 digit pertama dari kolom NAICS dan menghilangkan value 0

In [8]:
# Mengambil 2 digit pertama
naics = []
for item in df['NAICS']:
    if item == 0 :
        naics.append(0)
    else :
        a = list(str(item))[:2]
        b = ''.join(a)
        naics.append(b)

df['NAICS'] = naics


# Drop 0
index_naics = []
for item in list(df[df['NAICS'] == 0].index):
    index_naics.append(item)

df.drop(index_naics,inplace=True)

In [9]:
# Mengganti 2 digit dari NAICS menjadi nama sektor industri terkait
df['NAICS'] = df['NAICS'].map({
    '11': 'Agriculture / Forestry / Fishing / Hunting',
    '21': 'Mining / Quarrying / Oil&Gas Extraction',
    '22': 'Utilities',
    '23': 'Construction',
    '31': 'Manufacturing',
    '32': 'Manufacturing',
    '33': 'Manufacturing',
    '42': 'Wholesale trade',
    '44': 'Retail trade',
    '45': 'Retail trade',
    '48': 'Transportation / Warehousing',
    '49': 'Transportation / Warehousing',
    '51': 'Information',
    '52': 'Finance / Insurance',
    '53': 'Real Estate / Rental / Leasing',
    '54': 'Proffesional / Scientific / Tech.Service',
    '55': 'Management of Companies and Enterprises',
    '56': 'Administrative & Support / Waste Management / Remediation Services',
    '61': 'Educational',
    '62': 'Health Care / Social Assistance',
    '71': 'Arts / Entertainment / Recreation',
    '72': 'Accomodation / Food Services',
    '81': 'Other Services (except public admin)',
    '92': 'Public Administration'
})

### Menghilangkan value selain 'Y' or 'N' pada column LowDoc, karena tidak terdefinisi dan tidak memiliki arti

In [10]:
index_drop_lowdoc = []
for index in list(df[(df['LowDoc'] != 'N') & (df['LowDoc'] != 'Y')].index):
    index_drop_lowdoc.append(index)

df.drop(index_drop_lowdoc,inplace=True)

## Data Cleansing

In [11]:
dfc_v1 = df.copy()

### Handle Missing Values

In [12]:
# Check column with missing values
dfc_v1.isnull().sum()

LoanNr_ChkDgt             0
Name                     11
City                      0
State                     8
Zip                       0
Bank                    622
BankState               625
NAICS                     0
ApprovalDate              0
ApprovalFY                0
Term                      0
NoEmp                     0
NewExist                132
CreateJob                 0
RetainedJob               0
FranchiseCode             0
UrbanRural                0
RevLineCr              2207
LowDoc                    0
ChgOffDate           548399
DisbursementDate       1914
DisbursementGross         0
BalanceGross              0
MIS_Status             1708
ChgOffPrinGr              0
GrAppv                    0
SBA_Appv                  0
dtype: int64

In [13]:
# Check column with 0 values
dfc_v1.eq(0).any()

LoanNr_ChkDgt        False
Name                 False
City                 False
State                False
Zip                   True
Bank                 False
BankState            False
NAICS                False
ApprovalDate         False
ApprovalFY           False
Term                  True
NoEmp                 True
NewExist              True
CreateJob             True
RetainedJob           True
FranchiseCode         True
UrbanRural            True
RevLineCr            False
LowDoc               False
ChgOffDate           False
DisbursementDate     False
DisbursementGross     True
BalanceGross          True
MIS_Status           False
ChgOffPrinGr          True
GrAppv               False
SBA_Appv             False
dtype: bool

In [14]:
 # Mengatasi missing value pada column MIS_Status
dfc_v1['MIS_Status'] = np.where(dfc_v1['ChgOffPrinGr'] == 0, 'P I F', 'CHGOFF')

In [15]:
# Mengatasi column Term, NoEmp, NewExist, UrbanRural yang memiliki nilai 0

# Term
term = []
for item in list(dfc_v1[dfc_v1['Term'] == 0].index):
    term.append(item)

dfc_v1.drop(term,inplace=True)

# NoEmp
noemp = []
for item in list(dfc_v1[dfc_v1['NoEmp'] == 0].index):
    noemp.append(item)

dfc_v1.drop(noemp,inplace=True)

# NewExist
newexist = []
for item in list(dfc_v1[dfc_v1['NewExist'] == 0].index):
    newexist.append(item)

dfc_v1.drop(newexist,inplace=True)

# UrbanRural
urbanrural = []
for item in list(dfc_v1[dfc_v1['UrbanRural'] == 0].index):
    urbanrural.append(item)

dfc_v1.drop(urbanrural,inplace=True)

### Handle Duplicated Data

In [17]:
dfc_v1.duplicated().sum()

0

> Tidak ada data yang duplicated, sehingga tidak perlu melakukan apapun

### Handle Outliers

In [18]:
# Mengatasi outliers di NoEmp, karena jumlah maksimal NoEmp diantara 250 - 1500
dfc_v1 = dfc_v1[dfc_v1['NoEmp'] <= 1500]

In [19]:
# Mengatasi outliers di DisbursementGross, menggunakan data yang value DisbursementGross < GrAppv & DisbursementGross == GrAppv
dfc_v1 = dfc_v1[dfc_v1['DisbursementGross'] <= dfc_v1['GrAppv']]

### Feature Transformation

### Feature Encoding

In [20]:
# Mengubah value NewExist
def change_newexist(x):
    # Value 1 akan diubah menjadi 1 (Existing business)
    if x == 1:
        return 1
    # Value 2 akan diubah menjadi 0 (New business)
    else:
        return 0
dfc_v1['NewExist'] = dfc_v1['NewExist'].apply(lambda x : change_newexist(x))

In [21]:
# Mengubah value FranchiseCode
def change_franchise(x):
    # Value 1 atau 0 akan diubah menjadi 0 (No Franchise)
    if (x == 1) | (x == 0):
        return 0
    # Value selain dari 1 atau 0 akan diubah menjadi 1 (Have Franchise)
    else :
        return 1

dfc_v1['FranchiseCode'] = dfc_v1['FranchiseCode'].apply(lambda x: change_franchise(x))

In [22]:
# Mengubah value UrbanRural
def change_urbanrural(x):
    # Value 1 akan diubah menjadi 1 (Urban)
    if x == 1:
        return 1
    # Value 2 akan diubah menjadi 0 (Rural)
    else:
        return 0

dfc_v1['UrbanRural'] = dfc_v1['UrbanRural'].apply(lambda x : change_urbanrural(x))

In [23]:
# Mengubah value LowDoc
def change_lowdoc(x):
    # Value N akan diubah menjadi 0
    if x == 'N':
        return 0
    # Value Y akan diubah menjadi 1
    else :
        return 1

dfc_v1['LowDoc'] = dfc_v1['LowDoc'].apply(lambda x: change_lowdoc(x))

In [24]:
# Mengubah value MIS_Status (PIF = 0, CHGOFF = 1)
dfc_v1['MIS_Status'] = dfc_v1['MIS_Status'].replace({'P I F': 0, 'CHGOFF':1})

In [25]:
# One hot encoding NAICS & State
dfc_v1 = pd.get_dummies(dfc_v1, columns=['NAICS', 'State'], drop_first=True, dtype=int)

In [26]:
dfc_v1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392169 entries, 12 to 898660
Data columns (total 94 columns):
 #   Column                                                                    Non-Null Count   Dtype         
---  ------                                                                    --------------   -----         
 0   LoanNr_ChkDgt                                                             392169 non-null  int64         
 1   Name                                                                      392164 non-null  object        
 2   City                                                                      392169 non-null  object        
 3   Zip                                                                       392169 non-null  int64         
 4   Bank                                                                      392160 non-null  object        
 5   BankState                                                                 392160 non-null  object        


### Handle Class Imbalance

## Feature Engineering

### Feature Selection

In [27]:
dfc_v2 = dfc_v1.drop(['LoanNr_ChkDgt',
                      'Name',
                      'City',
                      'Zip',
                      'Bank',
                      'BankState',
                      'ApprovalDate',
                      'CreateJob',
                      'RetainedJob',
                      'RevLineCr',
                      'ChgOffDate',
                      'BalanceGross',
                      'ChgOffPrinGr'], 
                     axis=1)
dfc_v2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392169 entries, 12 to 898660
Data columns (total 81 columns):
 #   Column                                                                    Non-Null Count   Dtype         
---  ------                                                                    --------------   -----         
 0   ApprovalFY                                                                392169 non-null  int64         
 1   Term                                                                      392169 non-null  int64         
 2   NoEmp                                                                     392169 non-null  int64         
 3   NewExist                                                                  392169 non-null  int64         
 4   FranchiseCode                                                             392169 non-null  int64         
 5   UrbanRural                                                                392169 non-null  int64         


### Feature Extraction

In [28]:
# Membuat feature baru berdasarkan column Term (Membuat column RealEstate)
dfc_v2['RealEstate'] = 0
dfc_v2.loc[dfc_v2['Term'] >= 300, 'RealEstate'] = 1

In [55]:
# Membuat feature baru berdasarkan column DisbursmentDate (Membuat column Resesi)
list_resesi = []

for resesi in dfc_v2['DisbursementDate'].astype(str):
  if resesi >= '2007-12-01' and resesi <= '2009-06-30':
    resesi = 1
  else:
    resesi = 0

  list_resesi.append(resesi)

dfc_v2['Resesi'] = list_resesi

In [61]:
# Membuat feature baru berdasarkan column SBAAppv & Gr_Appv (Membuat column Portion)
dfc_v2['Portion'] = dfc_v2['SBA_Appv'] / dfc_v2['GrAppv']

In [63]:
dfc_v3 = dfc_v2.drop(['DisbursementDate',
                      'ApprovalFY'], 
                     axis=1)
dfc_v3.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 392169 entries, 12 to 898660
Data columns (total 82 columns):
 #   Column                                                                    Non-Null Count   Dtype  
---  ------                                                                    --------------   -----  
 0   Term                                                                      392169 non-null  int64  
 1   NoEmp                                                                     392169 non-null  int64  
 2   NewExist                                                                  392169 non-null  int64  
 3   FranchiseCode                                                             392169 non-null  int64  
 4   UrbanRural                                                                392169 non-null  int64  
 5   LowDoc                                                                    392169 non-null  int64  
 6   DisbursementGross                                  

### New Feature

1. Menambah feature **Default_Rate** dari masing masing **sektor industri**
2. Menambah feature **Default_Rate** dari masing masing **State**
3. Membagi feature **State** menjadi **Region** untuk mengurangi jumlah kolom
4. 